Organization:
- Work
  - 1 test: defining functions for part 1, testing on test input
  - 1 run: getting answer for part 1
  - 2 test: ...
  - 2 run: ...
- Utilities: functions I think might help parse general inputs
- Inputs: where I define the test (_t_) and problem (_s_) inputs

# Work

## 1 test

Converting SNAFU to a decimal is easy. The reverse takes more work, but here's the idea:
- Converting to base 5 and "downshifting" the characters (4 -> 2, 1 -> -, etc) gives something that's too small by the number 2...2 in base 5 (with the appropriate number of 2s)
- So start by adding 2...2 to the original number, and then convert to base 5 with the character downshift
- The trick is to choose the right number of digits for this 2...2
- This depends on the highest value digit of the original number when written in base 5, but here's a trick: have 2...2 use one more digit than the original number in base 5 (because if we use the same number of digits, carrying can give us a new place). And then at the end if the number starts with 0, drop that.

In [71]:
import math

In [72]:
# Helper functions to convert, giving decimals as integers and snafu as strings

snafu_to_dec_dict = {'2' : 2, '1' : 1, '0' : 0, '-' : -1, '=' : -2}
dec_to_snafu_downshift_dict = {4 : '2', 3 : '1', 2 : '0', 1 : '-', 0 : '='}

# This one is easy
def snafu_to_dec(snafu):
    dec = 0
    place = 1
    for i,char in enumerate(snafu[::-1]):
        dec += snafu_to_dec_dict[char] * place
        place *= 5
    return dec

# See above for how this works
def dec_to_snafu(dec):
    # One more than the number of digits required to express the number in base 5
    digits = math.floor(math.log(dec,5)) + 2
    
    # Add the 2...2 to the input
    # Using the general case of 222_5 = (444_5)/2 = (1000_5)/2
    x = dec + (5**digits - 1) // 2
    
    # Convert to base 5 with a downshift on the characters
    digits = []
    while x:
        digits.append(dec_to_snafu_downshift_dict[x%5])
        x = x // 5
    if digits[-1] == '0':
        digits.pop()
    snafu = ''.join(digits[::-1])
    
    return snafu

In [73]:
split(t)

['1=-0-2',
 '12111',
 '2=0=',
 '21',
 '2=01',
 '111',
 '20012',
 '112',
 '1=-1=',
 '1-12',
 '12',
 '1=',
 '122']

In [74]:
apply_func(split(t), [snafu_to_dec, dec_to_snafu])

['1=-0-2',
 '12111',
 '2=0=',
 '21',
 '2=01',
 '111',
 '20012',
 '112',
 '1=-1=',
 '1-12',
 '12',
 '1=',
 '122']

In [76]:
dec_to_snafu(sum(apply_func(split(t), snafu_to_dec)))

'2=-1=0'

## 1 run

In [77]:
dec_to_snafu(sum(apply_func(split(s), snafu_to_dec)))

'2---1010-0=1220-=010'

# Utilities

In [1]:
# Remove initial/final \n characters
def clean(s):
    return s[1:-1]

# Split at \n characters
# If there are \n\n characters, split into blocks too
def split(s, block_char = '\n\n', line_char = '\n'):
    out = [block.split(line_char) for block in clean(s).split(block_char)]
    if len(out) == 1:
        return out[0]
    else:
        return out

# Apply a function(s) to a list or "block" data (2-level list)
def apply_func(data, func, nested=False):
    if not isinstance(func, list):
        func = [func]
        
    def _func(x):
        for f in func:
            x = f(x)
        return x
        
    if nested:
        return [[_func(x) for x in block] for block in data]
    else:
        return [_func(x) for x in data]

# Split, parsing everything as ints
def split_int(s):
    return apply_func(split(s), int)

# Split, parsing everything as float
def split_float(s):
    return apply_func(split(s), float)

# Inputs

In [2]:
t = """
1=-0-2
12111
2=0=
21
2=01
111
20012
112
1=-1=
1-12
12
1=
122
"""

In [3]:
s = """
22-=0-1-=
1-==1200200=22-1==2=
2-01=--01==01
20-=
21=210=002
12=-0002
1=100-==0
1101
2==-=0=0110-
2=100122==--1-
2=1-2==2--1
1==0=2=0==1100
2=0==-2
1=1222202101--=
1101=2111-11=
1=11=2022-11-0=
1=2=1102-
1--21=1--=
11-
1001-10-
201222-1-2
122--2
2-
11--2-=20=11-2=0
1-000=1221==2-
1=-==-=2-0
121-
1--2-=---0=01
1=00=010-1-20
1=110=2
1110-=01--0-12--0
200022-
2=-=0=-1==2-=2222
20201
11-0
1==
1-=---2==20=01
2=2=-1
1==01-100
1-=----=--=
1-011--20=1-==2121
101---1=-1--0=21--2
221=0100--020=11
1=-121=2=-1-=
11210-
1=0=2=22-==-==-
2=2220210221-202
1--1=---00-11
2=0--
1=-2=-1==2=0
1=0=0=
122012210-=010=1
21-1-220=21-=-00
2-=2==0
1220
20
1=0=22=-21101
2-===0==1-210
1=-220=121=----2
21=0=-=0---0===1=--
202-22112---0-2=
10=0
11
102120--=0-021-121
100--221=
2--011=-=202
1=-
110=0-0-021-
121
1-0210-0
2-10=2
2=-0-1=00--22
121-=2
12-212110-022-2211
1=222==11=-=-01=10
200-0120=
1-00===12=-1=-0--
100-1
102-2201
10-=0-==2=2==2=0
12=01
1012-2
1=0-02=21=1==1
2=021-
1-001-
10202-00=12===11-1
1=
20=11=0112=
2--1=1-1121
2-2=---222
1=-001==-0-
1-=112-=
11011
1=2==1
1-=2=2201
2==222-==
101-0-=2
1-10-0212-0==0-10
1-2-1-=01100
121=0-12-2
20=0=0-0
22000-==0-1-2=0=0=
10=
2101102
1120===10=2-2=02-
1-=10--211
2===-01==1=--===-
1=21-201-==200=
1=-=2=10
220-120=-0-1010
12011
1-0=2
1---01
1=-=1=021=0=0=011
1=10-212-0
2=
10-0=-1=
1-21=22=1101000-1-
11=00222-=100--
"""